In [235]:
#imports
import math
import jax
import jax.numpy as jnp
from jax.experimental.ode import odeint
import numpy as np
from functools import partial


In [236]:
#Definicja Lagrangianu dla pojedynńczego wachadła liczonego w sposób analityczny
#Argumenty funkcji:
#  -q   - położenie ciała
#  -q_t - prędkość ciała
#  -m   - masa ciała
#  -l   - długość liny
#  -g   - wartość przyśpieszenia ziemskiego
#Zwracane: wartość Lagrangianu w danym momencie.
def lagrangian_analitical(q, q_t, m, l, g):
    #Eneriga kinetyczna
    Ek=0.5*m*l*q_t**2
    
    #Energia potencjalna
    Ep=m*g*(l-l*jnp.cos(q))
    
    #Lagrangian
    return Ek-Ep
  

#Funkcja ma za zadanie reprezenotwanie równania różniczkowego, wynikającego z rozwiązania
#równanie Eulera=Lagrange'a
#Argumenty funkcji:
#   -lagrangian - Funkcja opisująca Lagrangian. Funkcja musi być typu ,,callable'' żeby,
#                 pochodne Lagrangianu zostały policzone.
#   -state      - Wektor zawierający wartości początkowe położenia i prędkości
#   -empty      - Aby ta funkcja mogła zostać poprawnie wywoływana w dalszej części programu
#                 koniecznym było dodanie ,,pustego'' argumentu, które nie będzie pełnił żadnej
#                 funkcji, ale pozwoli na poprawną kompilację
#Zwracane: Wektor zawiercający przyśpieszenia oraz prędkości w danym momencie.
def equation_of_motion(lagrangian, state, empty=None):
    
    #Podział współżędnych na położenia i prędkości
    if len(state)==2:
      q=state[0]
      q_t=state[1]
    else:
      q, q_t = jnp.split(state,2)
    
    #Wyznaczenie przyśpieszenia przy użyciu lagrangianu (wersja wielowymiarowa)
    q_tt = jax.numpy.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t)) @ (
		jax.grad(lagrangian, 0)(q, q_t)
		- jax.jacfwd(jax.grad(lagrangian, 1), 0)(q, q_t) @ q_t)
    return jnp.concatenate([q_t, q_tt])
  
#Funkcja wyznaczająca równanie ruchu przy użyciu lagrangianui i solvera odeint.
#Argumenty funkcji:
#   -lagrangian     - Funkcja opisująca Lagrangian. Funkcja musi być typu ,,callable'' żeby,
#                   pochodne Lagrangianu zostały policzone przy wywołaniu funkcji equation_of_motion
#   -initial_state  - Wektor zawierający wartości początkowe położenia i prędkości
#   -**kwargs       - Parametry wywołania solvera odeint
#Zwracane: Wektor opisujący dalsze położenia ciała
def solve_lagrangian(lagrangian, initial_state, **kwargs):
  #Definicja równania różniczowego opisującego dynamikę układu
  #Aby equation_of_motion zostało przekazane do solvera jako funkcja,
  #a nie jako wynik jej wywołania, skorzystano z funkcji partial.
  equation = partial(equation_of_motion, lagrangian)
  
  #Rozwiązanie równania przy użyciu solvera
  return odeint(equation, initial_state, **kwargs)


#Funkcja wyznaczająca równanie ruchu przy użyciu Lagrangianu wyznaczonego
#w sposób analityczny
#Argumenty funkcji:
#   -initial_state - Początkowego położenie oraz prędkości
#   -times         - Czas wyznaczanej trajektori
#   -m             - masa wachadła
#   -l             - długość liny wachadła
#   -g             - przyśpieszenie ziemskie
def solve_autograd(initial_state, times, m=1, l=1, g=9.8):
  lagrangian = partial(lagrangian_analitical, m=m, l=l, g=g)
  return solve_lagrangian(lagrangian, initial_state, t=times)

In [237]:
x0 = np.array([3*np.pi/7, 0], dtype=np.float32)
noise = np.random.RandomState(0).randn(x0.size)
t = np.linspace(0, 40, num=401, dtype=np.float32)

In [242]:
solve_autograd(x0,t)

Array([[ 1.3463968 ,  7.        ,  0.        ,  0.        ],
       [ 1.2943919 ,  7.0106044 , -1.0476673 ,  0.2332048 ],
       [ 1.1341046 ,  7.054241  , -2.1737695 ,  0.68464863],
       ...,
       [ 0.51981485,  8.339424  , -0.47119275, -0.95004636],
       [ 0.45440513,  8.199368  , -0.80010355, -1.8659183 ],
       [ 0.36996794,  7.962736  , -0.8231459 , -2.887571  ]],      dtype=float32)